<a href="https://colab.research.google.com/github/cjvaugha/swiss_tweet_agenda_setting/blob/main/scripts/classify_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 3930 (delta 28), reused 42 (delta 11), pack-reused 3854
Receiving objects: 100% (3930/3930), 8.33 MiB | 15.16 MiB/s, done.
Resolving deltas: 100% (2445/2445), done.


In [ ]:
%cd fastText

/content/fastText


In [ ]:
!sudo pip install .

Processing /content/fastText
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3135469 sha256=bdce2c2c0e760bf5f748198d61824e20d8d895e773f88c7caa91927f15362887
  Stored in directory: /tmp/pip-ephem-wheel-cache-sbx5k3wy/wheels/22/04/6e/b3aba25c1a5845898b5871a0df37c2126cb0cc9326ad0c08e7
Successfully built fasttext


In [ ]:
import fasttext
import csv

In [ ]:
import numpy as np
import pandas as pd

from gensim.utils import simple_preprocess


#import the dataset
d = pd.read_csv('https://raw.githubusercontent.com/cjvaugha/swiss_tweet_agenda_setting/main/resources/train_test_tweets_translated.csv')[['label', 'tweet_text']].rename(columns = {'tweet_text': 'text', 'label': 'env_relev'})
dataset = d.iloc[:int(0.8 * 3677)] #training data
dataset = dataset.astype(str)
validation = d.iloc[int(0.8 * 3677):] #validation data
validation = validation.astype(str)


#NLP pre-process
dataset.iloc[:, 0] = dataset.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
validation.iloc[:, 0] = validation.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

# Prefixing each row of the category column with '__label__'
dataset.iloc[:, 0] = dataset.iloc[:, 0].apply(lambda x: '__label__' + x)
validation.iloc[:, 0] = validation.iloc[:, 0].apply(lambda x: '__label__' + x)

# Saving the CSV file as a text file to train/test the classifier
dataset[['env_relev', 'text']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

validation[['env_relev', 'text']].to_csv('test.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [ ]:
with open("test.txt") as file:
  f = file.read()
  print(f[:600])

__label__positive Gotthard  Young  Christian  Democrats  for  sustainable  renovation  of  the  Gotthard:  Clear  vote  in  favor  of  the  2nd  pipe
__label__positive rejette  le  system  incitatif  en  matire  climatique  et  nergtique.  OUI  la  SE2050!\n
__label__negative One  day  after  Alstom  and  one  year  after  the  franc  shock,  the  SNB  needs  clearer  objectives
__label__positive the  climate  and  the  relationship  with  the  EU  are  too  important  for  me  to  waste  valuable  time  with  poisonous  things,"  says  our  President  for  Climate  Protection,  Climate  Choic


In [ ]:
# Training the fastText classifier
model = fasttext.train_supervised('train.txt', wordNgrams = 2, epoch = 25)

In [ ]:
# Evaluating performance on the entire test file
model.test('test.txt')  

(736, 0.9402173913043478, 0.9402173913043478)

In [ ]:
model.predict("Many parties back down when it comes to implementing concrete, effective measures for more security. The SVP, on the other hand, works tirelessly to ensure the security of the people in our country.")

(('__label__negative',), array([0.96650487]))

In [ ]:
data_to_classify = pd.read_csv("https://raw.githubusercontent.com/cjvaugha/swiss_tweet_agenda_setting/main/resources/tweet_data_translated.csv")
texts = data_to_classify["tweet_text"]
texts = texts.astype(str)

In [ ]:
env_relev = []

for line in texts:
  text_class = model.predict(line)
 

  if text_class[0][0] == "__label__positive":
    env_relev.append("1")
  else:
    env_relev.append("0")


In [ ]:
data_to_classify["env_relev"] = env_relev

In [ ]:
dates_unfixed = np.arange(np.datetime64("2019-03-01 00:00"), np.datetime64("2019-04-01 00:00"), np.timedelta64(1,'h'), dtype='datetime64[h]')

dates = []

for d in dates_unfixed:
  dt = pd.to_datetime(d)
  dates.append(dt.strftime("%Y-%m-%d %H"))

print(dates)


['2019-03-01 00', '2019-03-01 01', '2019-03-01 02', '2019-03-01 03', '2019-03-01 04', '2019-03-01 05', '2019-03-01 06', '2019-03-01 07', '2019-03-01 08', '2019-03-01 09', '2019-03-01 10', '2019-03-01 11', '2019-03-01 12', '2019-03-01 13', '2019-03-01 14', '2019-03-01 15', '2019-03-01 16', '2019-03-01 17', '2019-03-01 18', '2019-03-01 19', '2019-03-01 20', '2019-03-01 21', '2019-03-01 22', '2019-03-01 23', '2019-03-02 00', '2019-03-02 01', '2019-03-02 02', '2019-03-02 03', '2019-03-02 04', '2019-03-02 05', '2019-03-02 06', '2019-03-02 07', '2019-03-02 08', '2019-03-02 09', '2019-03-02 10', '2019-03-02 11', '2019-03-02 12', '2019-03-02 13', '2019-03-02 14', '2019-03-02 15', '2019-03-02 16', '2019-03-02 17', '2019-03-02 18', '2019-03-02 19', '2019-03-02 20', '2019-03-02 21', '2019-03-02 22', '2019-03-02 23', '2019-03-03 00', '2019-03-03 01', '2019-03-03 02', '2019-03-03 03', '2019-03-03 04', '2019-03-03 05', '2019-03-03 06', '2019-03-03 07', '2019-03-03 08', '2019-03-03 09', '2019-03-03 1

In [ ]:
phold = []
d = {}

data_by_publisher = data_to_classify.loc[data_to_classify["publisher"] == "vertliberaux"]

for date in dates:
  for index, row in data_by_publisher.iterrows():
    if date in row["date"]:
      phold.append(row["env_relev"])

  results = list(map(int, phold))
  if (len(phold) != 0):
    d[date] = sum(results) / len(phold)
  else:
    d[date] = 0
  phold = []

In [ ]:
print(d)

{'2019-03-01 00': 0, '2019-03-01 01': 0, '2019-03-01 02': 0, '2019-03-01 03': 0, '2019-03-01 04': 0, '2019-03-01 05': 0, '2019-03-01 06': 0, '2019-03-01 07': 0.0, '2019-03-01 08': 0, '2019-03-01 09': 0, '2019-03-01 10': 0, '2019-03-01 11': 0, '2019-03-01 12': 0, '2019-03-01 13': 0, '2019-03-01 14': 0, '2019-03-01 15': 0, '2019-03-01 16': 0, '2019-03-01 17': 1.0, '2019-03-01 18': 0, '2019-03-01 19': 0, '2019-03-01 20': 0, '2019-03-01 21': 0, '2019-03-01 22': 0, '2019-03-01 23': 0, '2019-03-02 00': 0, '2019-03-02 01': 0, '2019-03-02 02': 0, '2019-03-02 03': 0, '2019-03-02 04': 0, '2019-03-02 05': 0, '2019-03-02 06': 0, '2019-03-02 07': 0, '2019-03-02 08': 0, '2019-03-02 09': 0, '2019-03-02 10': 0, '2019-03-02 11': 0, '2019-03-02 12': 0, '2019-03-02 13': 0, '2019-03-02 14': 0, '2019-03-02 15': 0, '2019-03-02 16': 0, '2019-03-02 17': 0, '2019-03-02 18': 0, '2019-03-02 19': 0, '2019-03-02 20': 0, '2019-03-02 21': 0, '2019-03-02 22': 0, '2019-03-02 23': 0, '2019-03-03 00': 0, '2019-03-03 01'

In [ ]:
print(0:100000)

In [ ]:
print(100001:100001)

In [ ]:
model_fitted = model.fit(1)
model_fitted.summary()

AttributeError: ignored